# install the mne and braindecode librarie 

In [1]:
!pip install mne # for Open-source Python software for exploring, visualizing, 
                 # and analyzing human neurophysiological data: MEG, EEG, sEEG, ECoG, and more.
!pip install braindecode # A deep learning toolbox to decode raw time-domain EEG.

     |████████████████████████████████| 6.6MB 4.3MB/s 
     |████████████████████████████████| 327kB 6.1MB/s 
  Created wheel for braindecode: filename=Braindecode-0.4.85-cp36-none-any.whl size=71078 sha256=ce13480d954b46975750142eac7e4903cf0c1a168a7c21d72df9cb40a0bc740e
  Stored in directory: /root/.cache/pip/wheels/72/07/db/c243929cde1ab13cdb3da732b19244e93ed2c44a30894b8c4a
Successfully built braindecode


In [0]:
import mne
from mne.io import concatenate_raws
import numpy as np
import torch as th
from torch import nn
import torch.nn.functional as F
from braindecode.torch_ext.util import set_random_seeds
from braindecode.models.util import to_dense_prediction_model
from braindecode.datautil.iterators import CropsFromTrialsIterator
from braindecode.torch_ext.util import np_to_var, var_to_np
from braindecode.experiments.monitors import compute_preds_per_trial_from_crops
from braindecode.torch_ext.optimizers import AdamW
from braindecode.torch_ext.schedulers import ScheduledOptimizer, CosineAnnealing
from braindecode.datautil.iterators import get_balanced_batches
from braindecode.datautil.signal_target import SignalAndTarget
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from numpy.random import RandomState


In [3]:
# (open and close the left or right fist)               ---> 3,7,11 

# (imagine opening and closing the left or right fist)  ---> 4,8,12 <--- I chose this

# (open and close both fists or both feet)              ---> 5,9,13

# (imagine opening and closing both fists or both feet) ---> 6,10,14

# subject_id = [] # carefully cherry-picked to give nice results on such limited data :)
# This will download the files if you don't have them yet,and then return the paths to the files.
physionet_paths = [mne.datasets.eegbci.load_data(sub_id,[4,8,12]) for sub_id in range(1,80)]
physionet_paths = np.concatenate(physionet_paths)

# Load each of the files
parts = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto',verbose='WARNING')
         for path in physionet_paths]

# Concatenate them
raw = concatenate_raws(parts)

# Find the events in this dataset
events, _ = mne.events_from_annotations(raw) #<--- no use the event_id for that is the "_"

# Use only EEG channels
eeg_channel_inds = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Each annotation includes one of three codes (T0=1, T1=2, or T2=3):

# T0 corresponds to rest
# T1 corresponds to onset of motion (real or imagined) of
# the left fist (in runs 3, 4, 7, 8, 11, and 12)
# both fists (in runs 5, 6, 9, 10, 13, and 14)
# T2 corresponds to onset of motion (real or imagined) of
# the right fist (in runs 3, 4, 7, 8, 11, and 12)
# both feet (in runs 5, 6, 9, 10, 13, and 14)

# Extract trials, only using EEG channels
epoched = mne.Epochs(raw, events, dict(left=2, right=3), tmin=1, tmax=4.1, proj=False, picks=eeg_channel_inds,
                baseline=None, preload=True)

Using default location ~/mne_data for EEGBCI...
Creating ~/mne_data



Do you want to set the path:
    /root/mne_data
as the default EEGBCI dataset path in the mne-python config [y]/n? y
Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json



Used Annotations descriptions: ['T0', 'T1', 'T2']
3555 matching events found
No baseline correction applied
Not setting metadata
Loading data for 3555 events and 497 original time points ...
178 bad epochs dropped


In [0]:
# Convert data from volt to millivolt
# Pytorch expects float32 for input and int64 for labels.
X = (epoched.get_data() * 1e6).astype(np.float32)
y = (epoched.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1 

In [5]:
epoched

<Epochs  |   3377 events (all good), 1 - 4.1 sec, baseline off, ~819.6 MB, data loaded,
 'left': 1697
 'right': 1680>

In [6]:
epoched.events[:,2]-2 #---> the 0 represent the left and the 1 represent the right 

array([1, 0, 0, ..., 1, 0, 1])

In [0]:
train_set = SignalAndTarget(X[:2500], y=y[:2500]) 
valid_set = SignalAndTarget(X[2700:3000], y=y[2700:3000]) 

In [8]:
train_set.X.shape[2]

497

In [9]:
train_set.X.shape[1]

64

In [0]:
# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.

cuda = th.cuda.is_available()
set_random_seeds(seed=20170629, cuda=cuda)

# This will determine how many crops are processed in parallel
input_time_length = train_set.X.shape[2]
n_classes = 2
in_chans = train_set.X.shape[1]

# final_conv_length determines the size of the receptive field of the ConvNet
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes, 
                        input_time_length=input_time_length,final_conv_length = "auto").create_network()
to_dense_prediction_model(model)

if cuda:
    model.cuda()

In [14]:
cuda

True

In [16]:
# determine output size
test_input = np_to_var(np.ones((2, in_chans, input_time_length, 1), dtype=np.float32))
if cuda:
    test_input = test_input.cuda()
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]
print("{:d} predictions per input/trial".format(n_preds_per_input))

9 predictions per input/trial


In [0]:
# The iterator has the method get_batches, which can be used to get randomly shuffled training batches 
# with shuffle=True or ordered batches (i.e. first from trial 1, then from trial 2, etc.) with shuffle=False
iterator = CropsFromTrialsIterator(batch_size=64,input_time_length=input_time_length,
                                  n_preds_per_input=n_preds_per_input)

In [0]:
# Diferents values for the optimizer, uncoment and try ;) 
#optimizer = AdamW(model.parameters(), lr=2*0.01, weight_decay=0.01) # these are good values for the deep model
#optimizer = AdamW(model.parameters(), lr=1*0.001, weight_decay=0.0001)
optimizer = AdamW(model.parameters(), lr=0.06255 * 0.015, weight_decay=0)
#optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
#optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)

# Need to determine number of batch passes per epoch for cosine annealing

n_updates_per_epoch = len([None for b in iterator.get_batches(train_set, True)])
n_epochs = 50
scheduler = CosineAnnealing(n_epochs * n_updates_per_epoch)

# schedule_weight_decay must be True for AdamW
optimizer = ScheduledOptimizer(scheduler, optimizer, schedule_weight_decay=True)

In [19]:
#lets go to train !!! 
for i_epoch in range(50):
    # Set model to training mode
    model.train()
    for batch_X, batch_y in iterator.get_batches(train_set, shuffle=True):
        net_in = np_to_var(batch_X)
        if cuda:
            net_in = net_in.cuda()
        net_target = np_to_var(batch_y)
        if cuda:
            net_target = net_target.cuda()
        # Remove gradients of last backward pass from all parameters
        optimizer.zero_grad()
        outputs = model(net_in)
        # Mean predictions across trial
        # Note that this will give identical gradients to computing
        # a per-prediction loss (at least for the combination of log softmax activation
        # and negative log likelihood loss which we are using here)
        outputs = th.mean(outputs, dim=2, keepdim=False)
        loss = F.nll_loss(outputs, net_target)
        loss.backward()
        optimizer.step()

    # Print some statistics each epoch
    model.eval()
    print("Epoch {:d}".format(i_epoch))
    for setname, dataset in (('Train', train_set),('Valid', valid_set)):
        # Collect all predictions and losses
        all_preds = []
        all_losses = []
        batch_sizes = []
        for batch_X, batch_y in iterator.get_batches(dataset, shuffle=False):
            net_in = np_to_var(batch_X)
            if cuda:
                net_in = net_in.cuda()
            net_target = np_to_var(batch_y)
            if cuda:
                net_target = net_target.cuda()
            outputs = model(net_in)
            all_preds.append(var_to_np(outputs))
            outputs = th.mean(outputs, dim=2, keepdim=False)
            loss = F.nll_loss(outputs, net_target)
            loss = float(var_to_np(loss))
            all_losses.append(loss)
            batch_sizes.append(len(batch_X))
        # Compute mean per-input loss
        loss = np.mean(np.array(all_losses) * np.array(batch_sizes) /
                       np.mean(batch_sizes))
        print("{:6s} Loss: {:.5f}".format(setname, loss))
        # Assign the predictions to the trials
        preds_per_trial = compute_preds_per_trial_from_crops(all_preds,
                                                          input_time_length,
                                                          dataset.X)
        # preds per trial are now trials x classes x timesteps/predictions
        # Now mean across timesteps for each trial to get per-trial predictions
        meaned_preds_per_trial = np.array([np.mean(p, axis=1) for p in preds_per_trial])
        predicted_labels = np.argmax(meaned_preds_per_trial, axis=1)
        accuracy = np.mean(predicted_labels == dataset.y)
        print("{:6s} Accuracy: {:.1f}%".format(
            setname, accuracy * 100))

Epoch 0
Train  Loss: 0.75215
Train  Accuracy: 54.1%
Valid  Loss: 0.74894
Valid  Accuracy: 54.7%
Epoch 1
Train  Loss: 0.70677
Train  Accuracy: 56.4%
Valid  Loss: 0.74335
Valid  Accuracy: 55.0%
Epoch 2
Train  Loss: 0.63828
Train  Accuracy: 63.4%
Valid  Loss: 0.72425
Valid  Accuracy: 55.3%
Epoch 3
Train  Loss: 0.59209
Train  Accuracy: 67.5%
Valid  Loss: 0.70178
Valid  Accuracy: 59.3%
Epoch 4
Train  Loss: 0.56092
Train  Accuracy: 70.8%
Valid  Loss: 0.69322
Valid  Accuracy: 59.7%
Epoch 5
Train  Loss: 0.54016
Train  Accuracy: 72.6%
Valid  Loss: 0.64694
Valid  Accuracy: 60.0%
Epoch 6
Train  Loss: 0.50163
Train  Accuracy: 75.0%
Valid  Loss: 0.64687
Valid  Accuracy: 70.0%
Epoch 7
Train  Loss: 0.45885
Train  Accuracy: 78.0%
Valid  Loss: 0.61084
Valid  Accuracy: 71.0%
Epoch 8
Train  Loss: 0.65676
Train  Accuracy: 68.8%
Valid  Loss: 0.80049
Valid  Accuracy: 63.3%
Epoch 9
Train  Loss: 0.45315
Train  Accuracy: 80.6%
Valid  Loss: 0.60000
Valid  Accuracy: 67.3%
Epoch 10
Train  Loss: 0.47129
Train  Acc

In [20]:
# now we will test the model
test_set = SignalAndTarget(X[2500:2700], y=y[2500:2700])

model.eval()
# Collect all predictions and losses
all_preds = []
all_losses = []
batch_sizes = []
for batch_X, batch_y in iterator.get_batches(test_set, shuffle=False):
    net_in = np_to_var(batch_X)
    if cuda:
        net_in = net_in.cuda()
    net_target = np_to_var(batch_y)
    if cuda:
        net_target = net_target.cuda()
    outputs = model(net_in)
    all_preds.append(var_to_np(outputs))
    outputs = th.mean(outputs, dim=2, keepdim=False)
    loss = F.nll_loss(outputs, net_target)
    loss = float(var_to_np(loss))
    all_losses.append(loss)
    batch_sizes.append(len(batch_X))
# Compute mean per-input loss
loss = np.mean(np.array(all_losses) * np.array(batch_sizes) /
               np.mean(batch_sizes))
print("Test Loss: {:.5f}".format(loss))
# Assign the predictions to the trials
preds_per_trial = compute_preds_per_trial_from_crops(all_preds,
                                                  input_time_length,
                                                  test_set.X)
# preds per trial are now trials x classes x timesteps/predictions
# Now mean across timesteps for each trial to get per-trial predictions
meaned_preds_per_trial = np.array([np.mean(p, axis=1) for p in preds_per_trial])
predicted_labels = np.argmax(meaned_preds_per_trial, axis=1)
accuracy = np.mean(predicted_labels == test_set.y)
print("Test Accuracy: {:.1f}%".format(accuracy * 100))

Test Loss: 0.58392
Test Accuracy: 72.5%


In [21]:
# print the model arquitecture 
model

Sequential(
  (dimshuffle): Expression(expression=_transpose_time_to_spat)
  (conv_time): Conv2d(1, 40, kernel_size=(25, 1), stride=(1, 1))
  (conv_spat): Conv2d(40, 40, kernel_size=(1, 64), stride=(1, 1), bias=False)
  (bnorm): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_nonlin): Expression(expression=square)
  (pool): AvgPool2d(kernel_size=(75, 1), stride=(1, 1), padding=0)
  (pool_nonlin): Expression(expression=safe_log)
  (drop): Dropout(p=0.5, inplace=False)
  (conv_classifier): Conv2d(40, 2, kernel_size=(27, 1), stride=(1, 1), dilation=(15, 1))
  (softmax): LogSoftmax()
  (squeeze): Expression(expression=_squeeze_final_output)
)

# re training?
### lets go!!! 


In [25]:
# load the data
physionet_paths_v2 = [mne.datasets.eegbci.load_data(sub_id,[4,8,12]) for sub_id in range(81,108)]
physionet_paths_v2 = np.concatenate(physionet_paths_v2)

In [26]:
# Load each of the files
parts_v2 = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto',verbose='WARNING')
         for path in physionet_paths_v2]

# Concatenate them
raw_v2 = concatenate_raws(parts)

# Find the events in this dataset
events_v2, _ = mne.events_from_annotations(raw_v2) #<--- no use the event_id for that is the "_"

# Use only EEG channels
eeg_channel_inds_v2 = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

#Each annotation includes one of three codes (T0=1, T1=2, or T2=3):

#T0 corresponds to rest
#T1 corresponds to onset of motion (real or imagined) of
#the left fist (in runs 3, 4, 7, 8, 11, and 12)
#both fists (in runs 5, 6, 9, 10, 13, and 14)
#T2 corresponds to onset of motion (real or imagined) of
#the right fist (in runs 3, 4, 7, 8, 11, and 12)
#both feet (in runs 5, 6, 9, 10, 13, and 14)

# Extract trials, only using EEG channels
epoched_2 = mne.Epochs(raw_v2, events_v2, dict(left=2, right=3), tmin=1, tmax=4.1, proj=False, picks=eeg_channel_inds_v2,
                baseline=None, preload=True)


<ipython-input-26-a06eeb1a2549>:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  for path in physionet_paths_v2]
<ipython-input-26-a06eeb1a2549>:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  for path in physionet_paths_v2]
<ipython-input-26-a06eeb1a2549>:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  for path in physionet_paths_v2]


Used Annotations descriptions: ['T0', 'T1', 'T2']
7095 matching events found
No baseline correction applied
Not setting metadata
Loading data for 7095 events and 497 original time points ...
414 bad epochs dropped


In [0]:
# Convert data from volt to millivolt
# Pytorch expects float32 for input and int64 for labels.
X = (epoched_2.get_data() * 1e6).astype(np.float32)
y = (epoched_2.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1

In [28]:
epoched_2

<Epochs  |   6681 events (all good), 1 - 4.1 sec, baseline off, ~1.58 GB, data loaded,
 'left': 3349
 'right': 3332>

In [29]:
epoched_2.events[:,2]-2 #---> the 0 represent the left and the 1 represent the right 

array([1, 0, 0, ..., 1, 0, 1])

In [0]:
train_set = SignalAndTarget(X[:2535], y=y[:2535]) # we have 3377 events and we will use 1636 for the training
valid_set = SignalAndTarget(X[2535:3100], y=y[2535:3100]) #and for valid we will use 2450-1636 = 814 

In [0]:
# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = torch.cuda.is_available()
set_random_seeds(seed=20170629, cuda=cuda)

# This will determine how many crops are processed in parallel
input_time_length = train_set.X.shape[2]
n_classes = 2
in_chans = train_set.X.shape[1]
# final_conv_length determines the size of the receptive field of the ConvNet
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes, input_time_length=input_time_length,
                        final_conv_length=8).create_network()
to_dense_prediction_model(model)

if cuda:
    model.cuda()

In [32]:
# determine output size
test_input = np_to_var(np.ones((2, in_chans, input_time_length, 1), dtype=np.float32))
if cuda:
    test_input = test_input.cuda()
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]
print("{:d} predictions per input/trial".format(n_preds_per_input))

294 predictions per input/trial


In [0]:
iterator = CropsFromTrialsIterator(batch_size=64,input_time_length=input_time_length,
                                  n_preds_per_input=n_preds_per_input)

optimizer = AdamW(model.parameters(), lr=1*0.001, weight_decay=0.0001)


In [0]:
# Need to determine number of batch passes per epoch for cosine annealing

n_updates_per_epoch = len([None for b in iterator.get_batches(train_set, True)])
n_epochs = 50
scheduler = CosineAnnealing(n_epochs * n_updates_per_epoch)

# schedule_weight_decay must be True for AdamW
optimizer = ScheduledOptimizer(scheduler, optimizer, schedule_weight_decay=True)

In [35]:
# lets go to train again xD!!!
# but no with the same data  
for i_epoch in range(50):
    # Set model to training mode
    model.train()
    for batch_X, batch_y in iterator.get_batches(train_set, shuffle=True):
        net_in = np_to_var(batch_X)
        if cuda:
            net_in = net_in.cuda()
        net_target = np_to_var(batch_y)
        if cuda:
            net_target = net_target.cuda()
        # Remove gradients of last backward pass from all parameters
        optimizer.zero_grad()
        outputs = model(net_in)
        # Mean predictions across trial
        # Note that this will give identical gradients to computing
        # a per-prediction loss (at least for the combination of log softmax activation
        # and negative log likelihood loss which we are using here)
        outputs = th.mean(outputs, dim=2, keepdim=False)
        loss = F.nll_loss(outputs, net_target)
        loss.backward()
        optimizer.step()

    # Print some statistics each epoch
    model.eval()
    print("Epoch {:d}".format(i_epoch))
    for setname, dataset in (('Train', train_set),('Valid', valid_set)):
        # Collect all predictions and losses
        all_preds = []
        all_losses = []
        batch_sizes = []
        for batch_X, batch_y in iterator.get_batches(dataset, shuffle=False):
            net_in = np_to_var(batch_X)
            if cuda:
                net_in = net_in.cuda()
            net_target = np_to_var(batch_y)
            if cuda:
                net_target = net_target.cuda()
            outputs = model(net_in)
            all_preds.append(var_to_np(outputs))
            outputs = th.mean(outputs, dim=2, keepdim=False)
            loss = F.nll_loss(outputs, net_target)
            loss = float(var_to_np(loss))
            all_losses.append(loss)
            batch_sizes.append(len(batch_X))
        # Compute mean per-input loss
        loss = np.mean(np.array(all_losses) * np.array(batch_sizes) /
                       np.mean(batch_sizes))
        print("{:6s} Loss: {:.5f}".format(setname, loss))
        # Assign the predictions to the trials
        preds_per_trial = compute_preds_per_trial_from_crops(all_preds,
                                                          input_time_length,
                                                          dataset.X)
        # preds per trial are now trials x classes x timesteps/predictions
        # Now mean across timesteps for each trial to get per-trial predictions
        meaned_preds_per_trial = np.array([np.mean(p, axis=1) for p in preds_per_trial])
        predicted_labels = np.argmax(meaned_preds_per_trial, axis=1)
        accuracy = np.mean(predicted_labels == dataset.y)
        print("{:6s} Accuracy: {:.1f}%".format(
            setname, accuracy * 100))

Epoch 0
Train  Loss: 0.70229
Train  Accuracy: 52.3%
Valid  Loss: 0.70803
Valid  Accuracy: 52.6%
Epoch 1
Train  Loss: 0.68293
Train  Accuracy: 56.4%
Valid  Loss: 0.68782
Valid  Accuracy: 53.1%
Epoch 2
Train  Loss: 0.67629
Train  Accuracy: 61.0%
Valid  Loss: 0.68859
Valid  Accuracy: 56.6%
Epoch 3
Train  Loss: 0.64473
Train  Accuracy: 63.3%
Valid  Loss: 0.66717
Valid  Accuracy: 60.2%
Epoch 4
Train  Loss: 0.63687
Train  Accuracy: 65.2%
Valid  Loss: 0.64546
Valid  Accuracy: 66.2%
Epoch 5
Train  Loss: 0.61175
Train  Accuracy: 67.7%
Valid  Loss: 0.62634
Valid  Accuracy: 68.8%
Epoch 6
Train  Loss: 0.58978
Train  Accuracy: 72.0%
Valid  Loss: 0.61321
Valid  Accuracy: 70.3%
Epoch 7
Train  Loss: 0.57195
Train  Accuracy: 72.0%
Valid  Loss: 0.58979
Valid  Accuracy: 72.2%
Epoch 8
Train  Loss: 0.63493
Train  Accuracy: 64.1%
Valid  Loss: 0.66456
Valid  Accuracy: 64.1%
Epoch 9
Train  Loss: 0.56127
Train  Accuracy: 75.3%
Valid  Loss: 0.59316
Valid  Accuracy: 72.0%
Epoch 10
Train  Loss: 0.53958
Train  Acc

In [0]:
# :( no improve much